In [273]:
import pandas as pd
import datetime
from datetime import date
import numpy as np

In [274]:
file_path = r"C:\Users\divyal\Desktop\projects\NOVA\ta_ts\23-E1TA-20210723-50.xlsm"
df_ts = pd.read_excel(file_path, engine='openpyxl', sheet_name='Timesheet', header=7, usecols='A:F,K,M', dtype='str').dropna(how='all')
df_emp_list = pd.read_excel(file_path, engine = 'openpyxl', sheet_name = "Employee List", usecols="B,C,F,G", dtype='str').dropna(how='all')
df_date = pd.read_excel(file_path, sheet_name='Timesheet', engine='openpyxl', usecols='B', header=1)

C:\Users\divyal\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Users\divyal\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\divyal\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\divyal\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Users\divyal\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [275]:
temp_col = df_date.columns
work_date = temp_col[0]
work_date = datetime.datetime.strftime(work_date, "%b %d,%Y")
work_date

'Jul 23,2021'

In [276]:
df_nova_upload = df_ts.merge(df_emp_list, left_on="Last name, first name", right_on="Last name, first name", how='left')

In [277]:
def plant_val(row):
    row_val = str(row["Last name, first name"])
    if row_val is not None or row_val != "nan":
        return 2115

def date_col(row, work_date):
    row_val = str(row["Last name, first name"])
    if row_val is not None or row_val != "nan":
        return f"({work_date})"

def conf_text(row):
    row_val = str(row["Last name, first name"])
    if row_val is not None or row_val != "nan":
        temp_name = row_val
        temp_name = temp_name.replace(", ", ";")
        date = row["Date Worked"]
        if row["Sub"] == "1" and row["Nights"] == "Yes":
            return f"{temp_name} {date} SUB;NS"
        elif row["Sub"] == "1":
            return f"{temp_name} {date} SUB"
        elif row["Nights"] == "Yes":
            return f"{temp_name} {date} NS"
        else:
            return f"{temp_name} {date}"

def char_count(row):
    row_val = str(row["Last name, first name"])
    if row_val is not None or row_val != "nan":
        return len(row["Confirmation Text"])

In [278]:
df_nova_upload['Plant'] = df_ts.apply(lambda row: plant_val(row), axis=1)

In [279]:
df_nova_upload['Date Worked'] = df_nova_upload.apply(lambda row: date_col(row, work_date), axis=1)

In [280]:
df_nova_upload['Confirmation Text'] = df_nova_upload.apply(lambda row: conf_text(row), axis=1)

In [281]:
df_nova_upload['Posting Date'] = date.today().strftime("%Y%m%d")

In [282]:
df_nova_upload

,"Last name, first name",Nights,Work order #,Op,Sub-op,Conf,Total Hours,Sub,Quinn #,OT Activity,WorkCentre,Plant,Date Worked,Confirmation Text,Posting Date
0,"ABDIRAHMAN, AHMED",NaN,6377492,20,NaN,NaN,10,1,353675,CLT,SCC,2115.0,"(Jul 23,2021)","ABDIRAHMAN;AHMED (Jul 23,2021) SUB",20210805
1,"ADAN, NUUR",NaN,6809477,20,NaN,NaN,10,1,354632,CLT,SCC,2115.0,"(Jul 23,2021)","ADAN;NUUR (Jul 23,2021) SUB",20210805
2,"AHMED, ABDIRIZAG",NaN,6834957,20,NaN,NaN,10,1,354631,CLT,SCC,2115.0,"(Jul 23,2021)","AHMED;ABDIRIZAG (Jul 23,2021) SUB",20210805
3,"ALARIC, BRIAN",NaN,6715439,20,NaN,NaN,10,0,5105311,CLT,SCC,2115.0,"(Jul 23,2021)","ALARIC;BRIAN (Jul 23,2021)",20210805
4,"ALARIC, LEONARD",NaN,6377492,20,NaN,NaN,5,0,3101754,CLT,SCC,2115.0,"(Jul 23,2021)","ALARIC;LEONARD (Jul 23,2021)",20210805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"VINAS, DOMINGO",NaN,6809477,20,NaN,NaN,4,0,354625,CLT,SCC,NaN,"(Jul 23,2021)","VINAS;DOMINGO (Jul 23,2021)",20210805
69,"WIGGINS, BLAKE",NaN,6798167,40,NaN,NaN,10,0,348115,CLT,SCC,NaN,"(Jul 23,2021)","WIGGINS;BLAKE (Jul 23,2021)",20210805
70,"WILEY, CLAY",NaN,6798167,40,NaN,NaN,10,0,354382,CLT,SCC,NaN,"(Jul 23,2021)","WILEY;CLAY (Jul 23,2021)",20210805
71,"YOUNG, JAMES",NaN,6798167,40,NaN,NaN,10,0,348136,CLT,SCC,NaN,"(Jul 23,2021)","YOUNG;JAMES (Jul 23,2021)",20210805


In [283]:
df_nova_upload.insert(15, ' ', '')
# df_nova_upload.insert(16, ' ', '')
df_nova_upload.insert(16, 'Character Count', '')
df_nova_upload['Character Count'] = df_nova_upload.apply(lambda row: char_count(row), axis=1)

In [284]:
df_nova_upload = df_nova_upload[['Work order #', 'Op', 'Sub-op', 'WorkCentre', 'Plant', 'Total Hours', 'OT Activity', 'Conf', 'Confirmation Text', 'Posting Date', 'Quinn #', 'Last name, first name', 'Date Worked', 'Sub', 'Nights']]

In [285]:
col_headers = ['WO',	'Operation',	'Sub-Op',	'Work Centre',	'Plant',	'Hours',	'Activity',	'Final CNF',	'Confirmation Text',	'Posting Date',	'Employee',	'Name',	'Date Worked',	'Sub',	'Nights']
df_nova_upload.columns = col_headers

In [286]:
df_nova_upload['Sub'] = df_nova_upload['Sub'].replace("1", "Sub")
df_nova_upload['Sub'] = df_nova_upload['Sub'].replace("0", "")

<ipython-input-286-7cff969ed418>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nova_upload['Sub'] = df_nova_upload['Sub'].replace("1", "Sub")
<ipython-input-286-7cff969ed418>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nova_upload['Sub'] = df_nova_upload['Sub'].replace("0", "")


In [287]:
df_nova_upload['Nights'] = df_nova_upload['Nights'].replace("Yes", "NS")

<ipython-input-287-50f620ff84b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nova_upload['Nights'] = df_nova_upload['Nights'].replace("Yes", "NS")


In [288]:
df_nova_upload.replace(np.NaN, "", inplace=True)
df_nova_upload.to_csv('nova_upload.csv', index=False)

C:\Users\divyal\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
